In [1]:
import os
os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"

In [2]:
def build_model(network, img_shape, deep_tuning = False):
    inputs = tf.keras.Input(shape=img_shape)

    # Selección del modelo base
    if network == 'vgg16':
        base_model = tf.keras.applications.VGG16(input_tensor=inputs,
                                                 include_top=False,
                                                 weights='imagenet')
    elif network == 'vgg19':
        base_model = tf.keras.applications.VGG19(input_tensor=inputs,
                                                 include_top=False,
                                                 weights='imagenet')
    elif network == 'resnet50':
        base_model = tf.keras.applications.ResNet50(input_tensor=inputs,
                                                    include_top=False,
                                                    weights='imagenet')
    elif network == 'inceptionv3':
        base_model = tf.keras.applications.InceptionV3(input_tensor=inputs,
                                                       include_top=False,
                                                       weights='imagenet')
    else:
        raise ValueError("Solo 'vgg16', 'vgg19', 'resnet50' o 'inceptionv3' son válidos.")

    base_model.trainable = deep_tuning

    # Adaptar arquitectura dependiendo de la red
    if network =='inceptionv3':
        x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
        x = tf.keras.layers.Dense(2048, activation='relu')(x)
        x = tf.keras.layers.Dropout(0.5)(x)
    elif network =='resnet50':
        x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
        x = tf.keras.layers.Dropout(0.5)(x)
    else:  # VGG16 y VGG19
        x = tf.keras.layers.Flatten()(base_model.output)
        x = tf.keras.layers.Dense(4096, activation='relu')(x)
        x = tf.keras.layers.Dense(4096, activation='relu')(x)
        x = tf.keras.layers.Dropout(0.2)(x)

    # Capa de salida
    outputs = tf.keras.layers.Dense(2, activation='softmax')(x)

    model = tf.keras.Model(inputs, outputs)
    # model.summary()
    return base_model, model

In [3]:
from tensorflow.keras import backend as K

# Función de preprocesado para redes VGG16, VGG19 y ResNet50
def preprocess_input1(x):
  data_format = K.image_data_format()
  assert data_format in {'channels_last', 'channels_first'}
  if data_format == 'channels_first':
      # 'RGB'->'BGR'
      x = x[::-1, :, :]
      # Zero-center by mean pixel

      x[0, :, :] -= 103.939
      x[1, :, :] -= 116.779
      x[2, :, :] -= 123.68
  else:
      # 'RGB'->'BGR'
      x = x[:, :, ::-1]
      # Zero-center by mean pixel
      x[:, :, 0] -= 103.939
      x[:, :, 1] -= 116.779
      x[:, :, 2] -= 123.68
  return x

# Función de preprocesado para redes InceptionV3 y Xception
def preprocess_input2(x):
  x /= 255.
  x -= 0.5
  x *= 2.
  return x

In [4]:
import os
import numpy as np
from glob import glob
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.vgg16 import preprocess_input
from PIL import Image

def load_images_from_folder(folder):
    image_paths = []
    labels = []
    class_names = ["normal", "unhealthy"]
    
    for class_name in class_names:
        class_folder = os.path.join(folder, class_name)
        if os.path.isdir(class_folder):
            images = glob(os.path.join(class_folder, "*.jpg")) + glob(os.path.join(class_folder, "*.png"))
            image_paths.extend(images)
            labels.extend([0 if class_name == "normal" else 1] * len(images))  # Healthy = 0, Estrabismo = 1
    
    return image_paths, labels

def load_image_array(path, img_size):
    return np.array(
        Image.open(path).convert("RGB").resize((img_size, img_size)),
        dtype=np.float32
    )

def set_data(network, img_size) :
    # Cargar los conjuntos de datos
    X_train, y_train = load_images_from_folder("kaggle-enrique-voluntarios/train")
    X_val, y_val = load_images_from_folder("kaggle-enrique-voluntarios/validation")
    X_test, y_test = load_images_from_folder("kaggle-enrique-voluntarios/test")

    # Sobrescribe X_* para que contengan arrays en lugar de rutas
    X_train = [load_image_array(p, img_size) for p in X_train if os.path.isfile(p)]
    X_val   = [load_image_array(p, img_size) for p in X_val   if os.path.isfile(p)]
    X_test  = [load_image_array(p, img_size) for p in X_test  if os.path.isfile(p)]

    # Preprocesar imágenes y filtrar errores
    if (network == 'vgg16' or network == 'vgg19' or network == 'resnet50') :
        print('Entrando en primer preprocesamiento')
        X_train_preprocessed = np.array([img for img in (preprocess_input1(img) for img in X_train) if img is not None])
        X_val_preprocessed   = np.array([img for img in (preprocess_input1(img) for img in X_val)   if img is not None])
        X_test_preprocessed  = np.array([img for img in (preprocess_input1(img) for img in X_test)  if img is not None])
    else :
        print('Entrando en segundo preprocesamiento')
        X_train_preprocessed = np.array([img for img in (preprocess_input2(img) for img in X_train) if img is not None])
        X_val_preprocessed   = np.array([img for img in (preprocess_input2(img) for img in X_val)   if img is not None])
        X_test_preprocessed  = np.array([img for img in (preprocess_input2(img) for img in X_test)  if img is not None])

    # Verificar si hay imágenes perdidas
    print(f"🔹 Total imágenes en X_train: {X_train_preprocessed.shape}")
    print(f"🔹 Total imágenes en X_val: {X_val_preprocessed.shape}")
    print(f"🔹 Total imágenes en X_test: {X_test_preprocessed.shape}")

    # Convertir etiquetas a formato one-hot
    num_classes = 2
    y_train = to_categorical(y_train, num_classes=num_classes)
    y_val = to_categorical(y_val, num_classes=num_classes)
    y_test = to_categorical(y_test, num_classes=num_classes)

    # Verificar formato final
    print(f"🔹 Etiquetas en formato one-hot (train): {y_train.shape}")
    print(f"🔹 Etiquetas en formato one-hot (val): {y_val.shape}")
    print(f"🔹 Etiquetas en formato one-hot (test): {y_test.shape}")
    
    return X_train_preprocessed, y_train, X_val_preprocessed, y_val, X_test_preprocessed, y_test, X_test

In [5]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau


def train(model, network, model_name, X_train_preprocessed, y_train, X_val_preprocessed, y_val, epochs_number, use_data_augmentation=False):
    # Callback para guardar el mejor modelo basado en la precisión de validación
    checkpoint = ModelCheckpoint(
        model_name,  # Nombre del archivo donde se guardará el mejor modelo
        monitor="val_accuracy",  # Monitoreamos la precisión en validación
        save_best_only=True,  # Guarda solo si es el mejor hasta el momento
        mode="max",  # Queremos la mayor precisión posible
        verbose=1  # Muestra mensajes cuando guarda un nuevo mejor modelo
    )
    
    early_stop = EarlyStopping(
        monitor="val_accuracy",  # O puedes usar 'val_loss' si prefieres
        patience=500,
        restore_best_weights=True,
        verbose=1
    )

    reduce_lr = ReduceLROnPlateau(
        monitor="val_loss",
        factor=0.5,
        patience=5,
        verbose=1,
        min_lr=1e-7
    )

    callbacks_list = [checkpoint]

    model.compile(optimizer=Adam(learning_rate=0.00001),  # cambio a 1e-4
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    if use_data_augmentation:
        # Definir el generador de imágenes con Data Augmentation
        datagen = ImageDataGenerator(
            brightness_range=(0.8, 1.2),  # Ajusta el brillo entre 80% y 120% del original
            # channel_shift_range=10.0,  # Cambia la intensidad de los colores en los canales RGB
            rotation_range=10,  # Rota la imagen aleatoriamente hasta 15 grados
            # zoom_range=0.1,  # Aplica zoom aleatorio en el rango de 80% a 120%
            horizontal_flip=True,  # Invierte la imagen horizontalmente con probabilidad del 50%
        )

        datagen.fit(X_train_preprocessed)

        # 4. Entrenamiento del modelo con `datagen.flow`
        history = model.fit(
            datagen.flow(X_train_preprocessed, y_train, batch_size=32),  # Generador de imágenes aumentado
            epochs=epochs_number,
            validation_data=(X_val_preprocessed, y_val),  # Validación sin aumento
            verbose=1,
            callbacks=callbacks_list,  # Usar el callback para guardar el modelo
        )
    else:
        # Entrenamiento sin Data Augmentation
        history = model.fit(
            X_train_preprocessed, y_train,  # Usa directamente los datos preprocesados
            batch_size=32,
            epochs=epochs_number,
            validation_data=(X_val_preprocessed, y_val),
            verbose=1,
            callbacks=[checkpoint]  # Agregar el callback aquí
        )

    return model, history

In [6]:
def show_metrics(history) :
    import matplotlib.pyplot as plt

    # Extraer métricas
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    accuracy = history.history['accuracy']
    val_accuracy = history.history['val_accuracy']

    epochs = range(1, len(loss) + 1)

    # Gráfica de Loss
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, loss, 'bo-', label='Training Loss')
    plt.plot(epochs, val_loss, 'ro-', label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Loss over Epochs')
    plt.legend()

    # Gráfica de Accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, accuracy, 'bo-', label='Training Accuracy')
    plt.plot(epochs, val_accuracy, 'ro-', label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title('Accuracy over Epochs')
    plt.legend()

    plt.show()

In [7]:
def evaluate_model(model, X_test_preprocessed, y_test) :
    # Evaluar el modelo
    test_loss, test_accuracy = model.evaluate(X_test_preprocessed, y_test, verbose=1)

    # Imprimir los resultados
    print(f"Loss en el conjunto de prueba: {test_loss:.4f}")
    print(f"Precisión en el conjunto de prueba: {test_accuracy:.4f}")

In [8]:
# Save the model
def save_model(model_name) :
    model.save(f"{model_name}.h5")
    model.save_weights(f"{model_name}.weights.h5")
    print(f"Saving model as {model_name}")

In [9]:
# import numpy as np
# import matplotlib.pyplot as plt
# import tensorflow as tf
# from tensorflow.keras.preprocessing import image
# from tensorflow.keras.models import Model
# import cv2

# def get_last_conv_layer_name(model):
#     for layer in reversed(model.layers):
#         if isinstance(layer, tf.keras.layers.Conv2D):
#             return layer.name
#     raise ValueError("No se encontró una capa Conv2D.")

# def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
#     grad_model = Model(
#         inputs=[model.inputs],
#         outputs=[model.get_layer(last_conv_layer_name).output, model.output]
#     )

#     with tf.GradientTape() as tape:
#         conv_outputs, predictions = grad_model(img_array)
#         if pred_index is None:
#             pred_index = tf.argmax(predictions[0])
#         class_channel = predictions[:, pred_index]

#     grads = tape.gradient(class_channel, conv_outputs)
#     pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

#     conv_outputs = conv_outputs[0]
#     heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
#     heatmap = tf.squeeze(heatmap)
#     heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
#     return heatmap.numpy()

# def save_and_display_gradcam(img_path, heatmap, alpha=0.4):
#     img = cv2.imread(img_path)
#     img = cv2.resize(img, (heatmap.shape[1], heatmap.shape[0]))
#     heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
#     heatmap = np.uint8(255 * heatmap)
#     heatmap_color = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
#     superimposed_img = cv2.addWeighted(img, 1 - alpha, heatmap_color, alpha, 0)
    
#     plt.imshow(cv2.cvtColor(superimposed_img, cv2.COLOR_BGR2RGB))
#     plt.axis('off')
#     plt.show()

In [10]:
# import os
# import numpy as np
# import cv2
# import tensorflow as tf
# import matplotlib.pyplot as plt
# import gc

# networks = ['vgg16', 'vgg19', 'resnet50', 'inceptionv3']
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
# tf.get_logger().setLevel('ERROR')

# for network in networks:
#     print(f"---------------{network}---------------")
    
#     img_size = 299 if network == 'inceptionv3' else 224
#     img_shape = (img_size, img_size, 3)
#     print(img_size)

#     base_model, model = build_model(network, img_shape)

#     X_train_preprocessed, y_train, X_val_preprocessed, y_val, X_test_preprocessed, y_test, X_test = set_data(network, img_size)
        
#     is_data_augmentation = True
#     model_name = f"{network}_kaggle_enrique_voluntarios_2"
#     model, history = train(model, network, f'{model_name}.keras', X_train_preprocessed, y_train, X_val_preprocessed, y_val, 200, is_data_augmentation)
    
#     model.load_weights(f'{model_name}.keras')

#     show_metrics(history)
#     evaluate_model(model, X_test_preprocessed, y_test)
#     save_model(model_name)

#     # Limpiar memoria
#     del model, base_model, history
#     del X_train_preprocessed, y_train, X_val_preprocessed, y_val, X_test_preprocessed, y_test
#     tf.keras.backend.clear_session()
#     gc.collect()


In [ ]:
import os
import numpy as np
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
import gc
# Deep Tuning
tf.keras.backend.clear_session()

networks = ['vgg16', 'vgg19']
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.get_logger().setLevel('ERROR')

for network in networks:
    print(f"---------------{network}---------------")
    
    img_size = 299 if network == 'inceptionv3' else 224
    img_shape = (img_size, img_size, 3)
    print(img_size)

    model_name = f"{network}_kaggle_enrique_voluntarios_2"
    base_model, model = build_model(network, img_shape, True)

    model.load_weights(f'{model_name}.keras')
    model_name = f"{network}_kaggle_enrique_voluntarios_2_deep_tuning"
    
    X_train_preprocessed, y_train, X_val_preprocessed, y_val, X_test_preprocessed, y_test, X_test = set_data(network, img_size)
        
    is_data_augmentation = True
    model, history = train(model, network, f'{model_name}.keras', X_train_preprocessed, y_train, X_val_preprocessed, y_val, 50, is_data_augmentation)
    
    model.load_weights(f'{model_name}.keras')

    show_metrics(history)
    evaluate_model(model, X_test_preprocessed, y_test)
    save_model(model_name)

    # Limpiar memoria
    del model, base_model, history
    del X_train_preprocessed, y_train, X_val_preprocessed, y_val, X_test_preprocessed, y_test
    tf.keras.backend.clear_session()
    gc.collect()

---------------vgg16---------------
224
Entrando en primer preprocesamiento
🔹 Total imágenes en X_train: (475, 224, 224, 3)
🔹 Total imágenes en X_val: (167, 224, 224, 3)
🔹 Total imágenes en X_test: (159, 224, 224, 3)
🔹 Etiquetas en formato one-hot (train): (475, 2)
🔹 Etiquetas en formato one-hot (val): (167, 2)
🔹 Etiquetas en formato one-hot (test): (159, 2)
Train for 15 steps, validate on 167 samples
Epoch 1/50
 1/15 [=>............................] - ETA: 3:31

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\diego\anaconda3\envs\env4\lib\site-packages\IPython\core\interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-11-a9c9deb4b8aa>", line 30, in <module>
    model, history = train(model, network, f'{model_name}.keras', X_train_preprocessed, y_train, X_val_preprocessed, y_val, 50, is_data_augmentation)
  File "<ipython-input-5-c0d490c953fa>", line 56, in train
    callbacks=callbacks_list,  # Usar el callback para guardar el modelo
  File "C:\Users\diego\anaconda3\envs\env4\lib\site-packages\tensorflow_core\python\keras\engine\training.py", line 819, in fit
    use_multiprocessing=use_multiprocessing)
  File "C:\Users\diego\anaconda3\envs\env4\lib\site-packages\tensorflow_core\python\keras\engine\training_v2.py", line 342, in fit
    total_epochs=epochs)
  File "C:\Users\diego\anaconda3\envs\env4\lib\site-packages\tensorflow_core\python\keras\engine\training_v2.py", line

In [11]:
# import gc
# import tensorflow as tf

# tf.get_logger().setLevel('ERROR')  # opcional
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# networks = ['vgg16', 'vgg19', 'resnet50', 'inceptionv3']
# for network in networks:
    
#     print(f"---------------{network}---------------")
    
#     img_size = 299 if network == 'inceptionv3' else 224
#     img_shape = (img_size, img_size, 3)

#     base_model, model = build_model(network, img_shape)

#     X_train_preprocessed, y_train, X_val_preprocessed, y_val, X_test_preprocessed, y_test, X_test = set_data(network, img_size)
        
#     is_data_augmentation = True
#     model_name = f"best_model_data_augmentation_kaggle_2_{network}" if is_data_augmentation == True else f"mejor_modelo_kaggle_2_{network}"
#     model, history = train(model, network, model_name, X_train_preprocessed, y_train, X_val_preprocessed, y_val, 5, is_data_augmentation)

#     show_metrics(history)
#     evaluate_model(model, X_test_preprocessed, y_test)
#     save_model(model_name)

#     # Limpiar memoria
#     del model, base_model, history
#     del X_train_preprocessed, y_train, X_val_preprocessed, y_val, X_test_preprocessed, y_test
#     tf.keras.backend.clear_session()
#     gc.collect()


ERROR! Session/line number was not unique in database. History logging moved to new session 1562
